In [35]:
import torch
from torch import autograd # 自动求导

x = torch.tensor(1.)
a = torch.tensor(1., requires_grad=True)
b = torch.tensor(2., requires_grad=True)
c = torch.tensor(3., requires_grad=True)

y = a**2*x + b*x + c

print('before:', a.grad, b.grad, c.grad)
grads = autograd.grad(y, [a, b, c])
print('after:', grads[0], grads[1], grads[2])

before: None None None
after: tensor(2.) tensor(1.) tensor(1.)


In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='./mnist/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./mnist/', 
                                          train=False, 
                                          transform=transforms.ToTensor())
print(train_dataset.train_data.size())
print(train_dataset.train_labels.size())
print(test_dataset.test_data.size())
print(test_dataset.test_labels.size())

# plt.imshow(train_dataset.train_data[1].numpy(),cmap = 'gray')
# plt.show()

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle= False)

# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLosss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
 
# Train the model
total_step = len(train_loader)
print("total_step:", total_step)

for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader): # images: [100, 1, 28, 28]
        images = images.reshape(-1, 28*28) # images: [100, 784]
        
        # Forward paa
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() # clear gradients for next train
        loss.backward() # backpropagation, compute gadients
        optimizer.step()
        
        if (i+1)%100 == 0:
            print('Epoch [{}/{}], step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


torch.Size([60000, 28, 28])
torch.Size([60000])
torch.Size([10000, 28, 28])
torch.Size([10000])
total_step: 600
Epoch [1/5], step [100/600], Loss: 2.2078
Epoch [1/5], step [200/600], Loss: 2.1230
Epoch [1/5], step [300/600], Loss: 2.0348
Epoch [1/5], step [400/600], Loss: 1.9849
Epoch [1/5], step [500/600], Loss: 1.8845
Epoch [1/5], step [600/600], Loss: 1.8411
Accuracy of the model on the 10000 test images: 68 %
